<a href="https://colab.research.google.com/github/MeidanGR/GooglePlayStore_Preprocessing_Python/blob/main/GooglePlayStore_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Play Store Data Preprocessing**
This data preprocessing progress has been established for an academic project of a BI course. The data will be analyzed within the project.

Major thanks to **Gautham Prakash** for publishing & maintenance this dataset, which you can find at https://www.kaggle.com/gauthamp10/google-playstore-apps


# **Google auth & libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# **CSV LOAD**


In [57]:
#Loading csv
df = pd.read_csv('/content/drive/My Drive/Google-Playstore.csv')

print("Dataframe original shape is", np.shape(df), end='\n\n')
print(df.info())
df.sample(2)

Dataframe original shape is (603047, 23)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 603047 entries, 0 to 603046
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   App Name           603046 non-null  object 
 1   App Id             603047 non-null  object 
 2   Category           603047 non-null  object 
 3   Rating             599287 non-null  float64
 4   Rating Count       599287 non-null  float64
 5   Installs           602875 non-null  object 
 6   Minimum Installs   602875 non-null  float64
 7   Maximum Installs   603047 non-null  int64  
 8   Free               603047 non-null  bool   
 9   Price              603047 non-null  float64
 10  Currency           602866 non-null  object 
 11  Size               603047 non-null  object 
 12  Minimum Android    602556 non-null  object 
 13  Developer Id       603045 non-null  object 
 14  Developer Website  423783 non-null  object 
 15  Developer

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Currency,Size,Minimum Android,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
406611,Onet Stars: Match & Connect Pairs,com.blackout.connect,Board,4.4,19.0,"1,000+",1000.0,1026,True,0.0,USD,44M,4.4 and up,Blackout Lab,https://blackout.barcelona/,blackoutlab.barcelona@gmail.com,"Jul 15, 2020",21/07/2020 4:58,Everyone,https://blackout.barcelona/privacy.php,True,False,False
136471,CS2 - Partner Edition,com.astragon.constructionsim2b2b,Simulation,NaN,NaN,"100,000+",100000.0,179700,True,0.0,USD,Varies with device,Varies with device,astragon Entertainment GmbH,NaN,supportcs2@astragon.de,NaN,07/12/2017 18:22,Everyone,http://www.bau-simulator.de/privacy-policy-cs2...,False,True,False


# **DATA FILTERING**
## Agenda: 
Data reduction down to ~100K rows, for a decent dataframe volume, shape, & a faster run in Google Data Studio.

## Guidelines:
- **Minimum Android** is the minimum android version which allowes operation for the app. The mose popular values were found: 4.1\4.0.3\6.0 and up & Varies with device.
- **Size** must exceed 1MB.
- **Currency** will be USD only.
- **Last updated** during 2020.


In [58]:
#Coloumn labels space fix
df.columns = [x.replace(" ", "_") for x in df.columns]

#Rows SELECT
df = df.query('(Minimum_Android == "4.1 and up" | Minimum_Android == "4.0.3 and up" | Minimum_Android == "Varies with device" | \
Minimum_Android == "6.0 and up") & not Size.str.endswith("k") & Currency == "USD" & Last_Updated.str.contains("2020")',engine='python')

#Coloumns FILTER
df = df.filter(items=['App_Name', 'Category', 'Rating', 'Rating_Count', 'Installs', 'Price', 'Size','Released','Last_Updated','Ad_Supported', 'In_App_Purchases'])

#Saving a copy of dataframe
dfcopy = df.copy(deep=True)

print("Dataframe reduced shape is", np.shape(df))
df.sample(10)

Dataframe reduced shape is (112092, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
121464,Clock for Home Screen,Personalization,4.2,6.0,500+,0.0,19M,"Sep 27, 2015",18/06/2020 15:05,True,True
559082,مسابقات الفائز تحدي ثقافي واسلامي,Word,4.6,101.0,"10,000+",0.0,Varies with device,"Dec 9, 2018",12/07/2020 1:44,True,False
565928,Unitáxi Sergipe,Travel & Local,0.0,0.0,5+,0.0,9.1M,"Apr 22, 2020",22/04/2020 20:09,False,False
516055,StudentIntra,Education,3.6,51.0,"10,000+",0.0,7.7M,NaN,03/02/2020 14:07,False,False
429375,Plevne Marşı Zil sesleri,Personalization,0.0,0.0,100+,0.0,5.8M,"Jun 7, 2020",12/06/2020 19:56,True,False
373644,Naija Music Download,Music & Audio,3.7,15.0,"5,000+",0.0,40M,"May 15, 2019",03/04/2020 2:46,True,False
38680,AgroSens,Business,4.6,167.0,"10,000+",0.0,11M,"May 25, 2017",09/04/2020 20:44,False,False
324043,LINE Clova,Tools,2.7,407.0,"100,000+",0.0,Varies with device,NaN,16/06/2020 10:44,False,False
401026,Oakland High School Patriots,Education,3.5,8.0,500+,0.0,5.1M,"Aug 1, 2018",16/06/2020 11:30,False,False
76414,BenditaTrinidad Brasil,Music & Audio,0.0,0.0,"1,000+",0.0,14M,"Jan 31, 2017",12/05/2020 10:17,True,False


# **CHECKPOINT DATA INFO**

In [59]:
print("Dataframe NULL count:")
print(df.isnull().sum(axis = 0), end="\n\n")

df.info()

Dataframe NULL count:
App_Name               1
Category               0
Rating              1880
Rating_Count        1880
Installs               0
Price                  0
Size                   0
Released            6338
Last_Updated           0
Ad_Supported           0
In_App_Purchases       0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112092 entries, 20 to 603046
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112091 non-null  object 
 1   Category          112092 non-null  object 
 2   Rating            110212 non-null  float64
 3   Rating_Count      110212 non-null  float64
 4   Installs          112092 non-null  object 
 5   Price             112092 non-null  float64
 6   Size              112092 non-null  object 
 7   Released          105754 non-null  object 
 8   Last_Updated      112092 non-null  object 
 9   Ad_Supported      112092 non-null  bool   
 

# **DATA CLEAN & CORRECTION**
- A null value that was found in **App_Name** are not acceptable for furthor analysis, also 4 apps named: "#NAME?" should be cleaned.
- The null values in **Rating, Rating_Count** will be replaced with mean values.

In [60]:
#NULL Extraction
df = df.query('not App_Name.isnull() & not App_Name=="#NAME?"',engine='python')

# Function replaces np.nan values in all of the data frame with relevant statistics \ dal7collab
def rep_missings(data_frame):
    d = data_frame.copy(deep = True)
    for c in data_frame:
        if (data_frame[c].dtype == 'float64'):
            if data_frame[c].isnull().values.any() :
                m = data_frame[c].dropna().mean()
                d[c].fillna(m, inplace = True)
    return d;

# Activating the function
df = rep_missings(df)

In [61]:
#'Size' coloumn excluding "Varies with device"
df['Size'] = df['Size'].replace(to_replace="Varies with device", 
                                   value = np.nan)

#Coloumn types correction
df['App_Name'] = df['App_Name'].astype({'App_Name': 'string'})
df['Category'] = df['Category'].astype({'Category': 'string'})
df['Rating'] = df['Rating'].astype({'Rating': 'float32'})
df['Rating_Count'] = df['Rating_Count'].astype({'Rating_Count': 'int32'})
df['Installs'] = df['Installs'].astype({'Installs': 'string'})
df['Price'] = df['Price'].astype({'Price': 'float32'})
df['Released'] = pd.to_datetime(df['Released']).dt.strftime('%Y-%m') #Fetching only yyyy-mm from the full datetime.
df['Last_Updated'] = pd.to_datetime(df['Last_Updated']).dt.strftime('%Y-%m') #Fetching only mm-yyyy from the full datetime.
df['Size'] = df['Size'].astype({'Size': 'string'})


print("Dataframe final shape is", np.shape(df))
df.sample(10)

Dataframe final shape is (112087, 11)


,App_Name,Category,Rating,Rating_Count,Installs,Price,Size,Released,Last_Updated,Ad_Supported,In_App_Purchases
336314,Mahjong Solitaire Mermaid,Board,4.4,387,"10,000+",0.0,4.3M,2017-09,2020-04,True,True
484949,SellaXtrading,Finance,3.2,93,"5,000+",0.0,62M,2015-07,2020-07,False,False
118826,City Hill Climb Car Simulator: GT Racing Car S...,Travel & Local,5.0,100,"5,000+",0.0,18M,2020-03,2020-06,True,True
161997,DMMブックス 電子書籍リーダー,Books & Reference,4.0,3230,"500,000+",0.0,<NA>,NaN,2020-07,False,False
254554,Hijab Wallpapers,Personalization,4.5,29,"5,000+",0.0,3.5M,2019-05,2020-08,True,False
402427,تلاوات نادرة للشيخ عبدالباسط بدون نت,Music & Audio,4.6,12,"1,000+",0.0,5.2M,2019-10,2020-03,True,False
186932,Electric Circuit Studio,Education,4.0,1303,"100,000+",0.0,23M,2019-06,2020-06,True,False
6055,2048 Jezzball,Arcade,4.3,181,"10,000+",0.0,1.5M,2016-01,2020-02,True,False
228186,Geek Call Recorder,Tools,4.0,40998,"1,000,000+",0.0,3.0M,2015-12,2020-02,True,False
428622,PlanMyDay,Productivity,4.2,458,"50,000+",0.0,<NA>,2012-03,2020-02,False,True


# **POSTPROCESSING DATA INFO**

In [62]:
#Genereal information after preprocessing
print(df.info(),end = '\n\n')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112087 entries, 20 to 602972
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   App_Name          112087 non-null  string 
 1   Category          112087 non-null  string 
 2   Rating            112087 non-null  float32
 3   Rating_Count      112087 non-null  int32  
 4   Installs          112087 non-null  string 
 5   Price             112087 non-null  float32
 6   Size              101269 non-null  string 
 7   Released          105749 non-null  object 
 8   Last_Updated      112087 non-null  object 
 9   Ad_Supported      112087 non-null  bool   
 10  In_App_Purchases  112087 non-null  bool   
dtypes: bool(2), float32(2), int32(1), object(2), string(4)
memory usage: 7.5+ MB
None



# **EXPORT TO GOOGLE DRIVE**


In [63]:
#Exporting a csv into google drive
#df.to_csv('/content/drive/My Drive/GooglePlay-Store_Optimized.csv')

# **STATISTICS**

In [65]:
#Retrieve coloumn general info 
name = 'App_Name'
print(df[name].describe(include=[object]))
print("Coloumn's real dtype is:", df[name].dtypes)

count                                   112087
unique                                  112087
top       弥生村を作ろう！稲刈りで全国統一 卑弥呼の時代に戦バトルで村・城下町育成
freq                                         1
Name: App_Name, dtype: object
Coloumn's real dtype is: string
